In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='https://fronteranorte.colef.mx/index.php/fronteranorte/issue/archive/2'
url_root= 'https://fronteranorte.colef.mx/index.php/fronteranorte/issue/archive/2'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('h2', class_='media-heading')
vol = [x.find('a').get('href')for x in volumen]
#vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('h3',class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
def get_url_items(sopa, url): 
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('h3',class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol2

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Frontera Norte'
    a='V. Ciencias Sociales'
    te='Estudios Sobre Fronteras, Territoriales y Regionales'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='download').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:188]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","Letter Submission","Carta de presentación de manuscritos","In memoriam","COVER","Code of Ethics","CODIGO DE ÉTICA","EDITORIAL PROCESS","In Memoriam","Complate journal","Preliminary","Preliminaries","Complete Journal","Página Legal","Gender Transitions Along Borders. The Northern Borderlands of Mexico and Morocco Marlene Solís",
"La democracia mexicana a nivel subnacional: De la deshegemonización al patrimonialismo","Thread of Blood",
"Virginia Garrard-Burnett and David Stoll (Ed.)","A la brava ése. Identidades juveniles en México: cholos, punks y chavos banda, México"
,"Narrativas etnográficas en la Sierra Tarahumara, México","Agua Caliente, el patio de recreo de las estrellas. Mafiosos, magnates y artistas de cine en el centro de esparcimiento más grande de América Paul J. Vanderwood"])]
df

In [ ]:
df1=df.drop(df_catalogo.index[[7,8,37,58]])
df1

In [ ]:
df1.to_csv('Revista140.1.csv', index=False)